The following exercises are meant to be solved by gathering the bash commands incrimentally in two scripts, one for ex 1.* the other for ex 2.* 

### Ex 1

1\.a Make a new directory called `students` in your home. Download a csv file with the list of students of this lab from [here](https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv) (use the `wget` command) and copy that to `students`. First check whether the file is already there

1\.b Make two new files, one containing the students belonging to PoD, the other to Physics.

1\.c For each letter of the alphabet, count the number of students whose surname starts with that letter. 

1\.d Find out which is the letter with most counts.

1\.e Assume an obvious numbering of the students in the file (first line is 1, second line is 2, etc.), group students "modulo 18", i.e. 1,19,37,.. 2,20,38,.. etc. and put each group in a separate file  

In [ ]:
#!/bin/bash

# 1.a Create a directory called "students" in your home directory
mkdir ~/students

# Check if the CSV file is already present in the "students" directory
if [ -e ~/students/students.csv ]; then
    echo "CSV file already exists."
else
    # Download the CSV file from the provided URL using wget
    wget -O ~/students/students.csv https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv
fi

# 1.b Extract students belonging to PoD and Physics into separate files
grep "PoD" ~/students/students.csv > ~/students/pod_students.csv
grep "Physics" ~/students/students.csv > ~/students/physics_students.csv

# 1.c Count students by the first letter of their surname
for letter in {A..Z}; do
    count=$(grep -c "^$letter" ~/students/students.csv)
    echo "$letter: $count"
done

# 1.d Find the letter with the most counts
max_count=0
max_letter=""
for letter in {A..Z}; do
    count=$(grep -c "^$letter" ~/students/students.csv)
    if [ $count -gt $max_count ]; then
        max_count=$count
        max_letter=$letter
    fi
done
echo "The letter with the most counts is $max_letter with $max_count students."

# 1.e Group students by "modulo 18" and save each group in separate files
for ((i = 1; i <= 18; i++)); do
    group_file="~/students/group_$i.csv"
    awk -v group="$i" 'NR % 18 == group' ~/students/students.csv > "$group_file"
done


### Ex 2

2.a Make a copy of the file `data.csv` removing the metadata and the commas between numbers; call it `data.txt`

2\.b How many even numbers are there?

2\.c Distinguish the entries on the basis of `sqrt(X^2 + Y^2 + Z^2)` is greater or smaller than `100*sqrt(3)/2`. Count the entries of each of the two groups 

2\.d Make `n` copies of data.txt (with `n` an input parameter of the script), where the i-th copy has all the numbers divided by i (with `1<=i<=n`).

In [ ]:
#!/bin/bash

# 2.a Make a copy of the file data.csv removing the metadata and the commas between numbers; call it data.txt
grep -E '^[0-9]' data.csv | sed 's/,//g' > data.txt

# 2.b How many even numbers are there?
even_count=$(awk '{ if ($1 % 2 == 0) count++ } END { print count }' data.txt)
echo "Number of even numbers: $even_count"

# 2.c Distinguish the entries based on sqrt(X^2 + Y^2 + Z^2) is greater or smaller than 100*sqrt(3)/2. Count the entries of each of the two groups
greater_than_threshold=0
smaller_than_threshold=0
threshold=$(echo "100 * sqrt(3) / 2" | bc -l)

while IFS= read -r line; do
    values=($line)
    x=${values[0]}
    y=${values[1]}
    z=${values[2]}
    distance=$(echo "sqrt($x^2 + $y^2 + $z^2)" | bc -l)
    if (( $(echo "$distance > $threshold" | bc -l) )); then
        ((greater_than_threshold++))
    else
        ((smaller_than_threshold++))
    fi
done < data.txt

echo "Entries with sqrt(X^2 + Y^2 + Z^2) > 100*sqrt(3)/2: $greater_than_threshold"
echo "Entries with sqrt(X^2 + Y^2 + Z^2) <= 100*sqrt(3)/2: $smaller_than_threshold"

# 2.d Make n copies of data.txt (with n an input parameter of the script), where the i-th copy has all the numbers divided by i (with 1<=i<=n).
n=$1
for i in $(seq 1 $n); do
    output_file="data_copy_$i.txt"
    awk -v divisor=$i '{ for (j = 1; j <= NF; j++) $j = $j / divisor; print }' data.txt > "$output_file"
    echo "Created $output_file with numbers divided by $i"
done
